##### Prototype — With Generative AI Tools

##### Importing the required libraries 

In [1]:
# (Generated with ChatGPT prompt: "Write the imports for a tabular ML regression pipeline with scikit-learn, XGBoost fallback, and visuals.")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, r2_score

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# Optional: XGBoost (GenAI suggested trying a boosted tree)
try:
    from xgboost import XGBRegressor
    HAS_XGB = True
except Exception:
    HAS_XGB = False
